# Analiza datelor
- ## Price - Coloana tinta
  - std_invprice - Are cea mai mare corelatie cu coloana tinta. 

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

dataset = pd.read_csv('Dataset.csv')
#print(dataset.head())
#dataset.info()
#dataset.describe()

#Convertam datele de tipul Object in bool(Adica 1 sau 0 care reprezinta tipul int)
dataset = pd.get_dummies(dataset, columns=['brand', 'full_model_name', 'model_name', 'fuel_type', 'city'])
#dataset.info()

corr_matrix = dataset.corr()
#corr_matrix

#target_correlation = corr_matrix['price'].abs().nlargest(2)
#target_correlation

## Amesticarea randurilor in dataset

In [20]:
dataset, columns = dataset.values, list(dataset.columns)
a = np.random.shuffle(dataset)
dataset = pd.DataFrame(dataset, columns=columns)

None


## New dataset (correlation 0.5-0.8)

In [16]:
interval_columns = corr_matrix[(corr_matrix['price'].abs() > 0.5) & (corr_matrix['price'].abs() < 0.8)].index

new_dataset = dataset[interval_columns]
#new_dataset

#new_dataset['price'] = dataset['price']
#new_dataset.corr()

## Taiarea dataset-ului cu sklearn

In [4]:
X = dataset.drop(columns=['Id', 'price'])
y = dataset['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Convertarea datelor in float din cauza ca dataframe-ul are doar elemente de tipul Object
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)

#Scoaterea la inf ori NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(X_train.mean(), inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.fillna(X_train.mean(), inplace=True)

#X_train.isnull().sum().sum()
#np.isinf(X_train).sum().sum()


# Antrenarea modelului din sklearn - Linear Regression

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Rezultatul la la antrenarea modelului

In [8]:
y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred, columns=['price'])
score = model.score(X_test, y_test)

#print(score)
y_pred

,price
0,2.445821e+05
1,-5.453040e+10
2,1.299938e+11
3,-6.066599e+10
4,3.758136e+10
...,...
513,9.073653e+04
514,6.598035e+05
515,2.747353e+10
516,2.382787e+06


## Taierea datasetului de coloane selectate cu sklearn train_test_split

In [304]:
X = new_dataset
y = dataset['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(X_train.mean(), inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

#X_train.isnull().sum()

# Antrenarea modelului

In [230]:
model2 = LinearRegression()
model2.fit(X_train, y_train)

LinearRegression()

## Resultatul

In [311]:
predict = pd.DataFrame(model2.predict(X_test), columns=['price'])
score = model2.score(X_test, y_test)

#predict
#score

# Implementarea Linear Regression de la zero din manual

In [236]:
import math

class LinearRegression():
    # Define a Linear Regression class to store our relevant functions in
    def __init__(self):
        '''
        Initializes the Linear Regression model.
        It also stores the self.params__ variable, which will be the weights that the model returns.
        '''
        self.params__ = None

    def gradientDescent(self, X, y, learning_rate=0.00005, iterations=500, batch_size=16):
        '''
        This function applies the Gradient Descent model to the dataset
        :param X: numpy.ndarray
        The X matrix containing the independent variable columns.
        :param y: numpy.ndarray
        The target vector y.
        '''
        # Add a column of ones for the constant term
        X = np.concatenate([X, np.ones_like(y)], axis=1)
        rows, cols = X.shape
        # Combine the X and y columns to more easily shuffle it later
        X = np.append(X, y, axis=1)
        # Make the initial random guess for w
        w = np.random.random((cols, 1))
        # Go through all the iterations
        for i in range(iterations):
            # Shuffle the rows of the data
            np.random.shuffle(X)
            # Define X and y again
            y_it = X[:, -1].reshape((rows, 1))
            X_it = X[:, :-1]
            # Iterate through the batches
            for batch in range(math.ceil(rows / batch_size)):
                batch_start = batch * batch_size
                # Cut a batch from the dataset
                x_batch = X_it[batch_start: min(batch_start + batch_size, X.shape[0])]
                y_batch = y_it[batch_start: min(batch_start + batch_size, X.shape[0])]
                # Subtract the gradient from our previous estimation
                w -= learning_rate * np.matmul(x_batch.transpose(), (np.matmul(x_batch, w) - y_batch))
        self.params__ = w
        return self

    def predict(self, X):
        X = np.concatenate([X, np.ones(X.shape[0]).reshape((X.shape[0], 1))], axis=1)
        return np.matmul(X, self.params__)

## Convertarea matricei y_train in 2 dimensionala

In [305]:
y_train = pd.DataFrame(y_train.transpose())
#y_train

## Antrenarea modelului

In [306]:
model3 = LinearRegression()
model3.gradientDescent(X_train, y_train)

## Resultatul

In [310]:
predict = pd.DataFrame(model3.predict(X_test), columns=['price'])
score = model2.score(X_test, y_test)

#predict
#score